In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pytesseract
!pip install keras-ocr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!apt install tesseract-ocr
!apt install libtesseract-dev
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2build2).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libtesseract-dev is already the newest version (4.1.1-2build2).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
image_path = '1.jpg'

In [ ]:
import cv2
import pytesseract
import os
import csv
from google.colab.patches import cv2_imshow
import keras_ocr
import shutil

In [ ]:
def mark_region(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    gradient = cv2.morphologyEx(gray, cv2.MORPH_GRADIENT, kernel)
    contours, _ = cv2.findContours(gradient, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    boxes = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        boxes.append((x, y, w, h))  # Store the box coordinates as (x, y, width, height)
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255), 2)
    return image, boxes

# Load the OCR pipeline
pipeline = keras_ocr.pipeline.Pipeline()

image_name = os.path.splitext(os.path.basename(image_path))[0]
os.makedirs('/content/drive/MyDrive/Image Project', exist_ok=True)
output_folder = os.path.join('/content/drive/MyDrive/Image Project', image_name)
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)
os.makedirs(output_folder, exist_ok=True)
text_file = os.path.join(output_folder, 'text.csv')
image_file = os.path.join(output_folder, 'image.csv')

with open(text_file, 'w', newline='') as text_file, open(image_file, 'w', newline='') as image_file:
    text_writer = csv.writer(text_file)
    image_writer = csv.writer(image_file)
    text_writer.writerow(['Text', 'X', 'Y', 'Width', 'Height'])
    image_writer.writerow(['Image', 'X', 'Y', 'Width', 'Height'])

    image, line_items_coordinates = mark_region(image_path)

    if line_items_coordinates:
        image_folder = os.path.join(output_folder, 'images')
        text_folder = os.path.join(output_folder, 'text')
        os.makedirs(image_folder, exist_ok=True)
        os.makedirs(text_folder, exist_ok=True)
        num = 1
        for idx, c in enumerate(line_items_coordinates):
            # Create a copy of the original image
            img_copy = image.copy()
            # Cropping image img = img_copy[y0:y1, x0:x1]
            img = img_copy[c[1]:c[1]+c[3], c[0]:c[0]+c[2]]
            # Convert the image to black and white for better OCR
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            ret, thresh1 = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY)
            # Save the detected text and coordinates to the CSV file
            text = pytesseract.image_to_string(thresh1, config='--psm 6')
            if len(text) > 25:
                img_path = os.path.join(image_folder, f'img{num}.jpg')
                cv2.imwrite(img_path, img)
                image_writer.writerow([f'img{num}.jpg', c[0], c[1], c[2], c[3]])
                # Perform OCR on the image
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                prediction_groups = pipeline.recognize([img])
                # Process the OCR results
                predictions = prediction_groups[0]
                num2 = 1
                for prediction in predictions:
                    text = prediction[0]
                    if len(text)>2:
                       bounding_box = prediction[1]
                       # Convert bounding box to 'X', 'Y', 'Width', 'Height' format
                       x = int(bounding_box[0][0])
                       y = int(bounding_box[0][1])
                       w = int(bounding_box[1][0] - bounding_box[0][0])
                       h = int(bounding_box[2][1] - bounding_box[1][1])
                       sub_box_img = img[y:y+h, x:x+w]
                       sub_box_path = os.path.join(text_folder, f'text{num}_{num2}.jpg')
                       if sub_box_img.size > 0:
                          cv2.imwrite(sub_box_path, sub_box_img)
                          # cv2_imshow(sub_box_img)
                          text_writer.writerow([f'text{num}_{num2}.jpg', x, y, w, h])
                          num2 += 1
                num += 1
    else:
        print("No line items found in the image.")

Looking for /root/.keras-ocr/craft_mlt_25k.h5
Looking for /root/.keras-ocr/crnn_kurapan.h5
1/1 [==============================] - 4s 4s/step


In [ ]:
import os
import csv
import math

# Load the CSV file
text_file = os.path.join(output_folder, 'text.csv')
with open(text_file, 'r') as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header row
    data = list(reader)

# Initialize merge groups dictionary
merge_groups = {}

# Set the maximum distance between images for merging
max_distance = 5

# Set the minimum distance between images for creating new merge group
min_distance_threshold = 150

# Iterate over the rows of the CSV data
for row in data:
    filename = row[0]
    # print(filename)
    group_found = False

    # Extract num from filename
    num = filename.split('_')[0].replace('text', '')

    # Check if the filename exists in any merge group
    for group_id, group_images in merge_groups.items():
        group_num = group_images[0].split('_')[0].replace('text', '')

        # If num matches group_num and the distance is within the threshold, add filename to the merge group
        if num == group_num:
            distances = []

            # Calculate the distance between the filename and all images in the merge group
            for image in group_images:
                # Find the corresponding coordinates for the images
                coordinate1 = None
                coordinate2 = None
                for row in data:
                    if row[0] == filename:
                        coordinate1 = [int(row[1]), int(row[2])]
                    elif row[0] == image:
                        coordinate2 = [int(row[1]), int(row[2])]

                    if coordinate1 is not None and coordinate2 is not None:
                        break

                # Calculate the Euclidean distance between the coordinates
                distance = math.sqrt((coordinate2[0] - coordinate1[0]) ** 2 + (coordinate2[1] - coordinate1[1]) ** 2)
                distances.append(distance)

            # Check if the minimum distance between the filename and all images in the merge group is greater than the threshold
            if min(distances) <= min_distance_threshold:
                group_images.append(filename)
                group_found = True
                break

    # If the filename doesn't exist in any merge group or the minimum distance condition is not satisfied, create a new merge group
    if not group_found:
        merge_groups[filename] = [filename]

# Calculate the distance between images in each merge group
merge_group_coordinates = {}
for group_images in merge_groups.values():
    group_coordinates = []

    # Find the corresponding coordinates for each image in the group
    for image in group_images:
        for row in data:
            if row[0] == image:
                coordinate = [int(row[1]), int(row[2]), int(row[1]) + int(row[3]), int(row[2]) + int(row[4])]
                group_coordinates.append(coordinate)
                break

    # Find the minimum bounding box for the merge group
    min_x = min(coordinate[0] for coordinate in group_coordinates)
    min_y = min(coordinate[1] for coordinate in group_coordinates)
    max_x = max(coordinate[2] for coordinate in group_coordinates)
    max_y = max(coordinate[3] for coordinate in group_coordinates)

    # Store the coordinate for the merge group using the first image filename as the key
    merge_group_coordinates[group_images[0]] = [min_x, min_y, max_x - min_x, max_y - min_y]

# Display the merge group coordinates
for group_id, coordinates in merge_group_coordinates.items():
    print(f"Group {group_id}: {coordinates}")


Group text1_1.jpg: [25, 218, 321, 88]
Group text2_1.jpg: [25, 221, 298, 85]
Group text3_1.jpg: [25, 273, 368, 100]
Group text3_2.jpg: [253, 273, 362, 60]
Group text3_4.jpg: [623, 273, 75, 26]


In [ ]:
text_file = os.path.join(output_folder, 'text.csv')
os.remove(text_file)
import shutil
import pandas as pd
shutil.rmtree(text_folder)
text_folder = os.path.join(output_folder,'text')
os.makedirs(text_folder, exist_ok=True)

In [ ]:
import pandas as pd
new_dict = {}
image_list = []
x_list = []
y_list = []
width_list = []
height_list = []
i = 0
for key, value in merge_group_coordinates.items():
    new_key = key.replace('text', 'img')
    new_key = new_key[:4]+'.jpg'
    new_dict[new_key] = value
    img_path = os.path.join(image_folder,new_key)
    img = cv2.imread(img_path)
    cropped_image = img[value[1]:value[1]+value[3],value[0]:value[0]+value[2]]
    save_image_name = f'text{i}.jpg'
    save_image_path = os.path.join(text_folder,save_image_name)
    print(save_image_path)
    cv2.imwrite(save_image_path,cropped_image)
    # Append the values to the respective lists
    image_list.append(save_image_name)
    x_list.append(value[0])
    y_list.append(value[1])
    width_list.append(value[2])
    height_list.append(value[3])
    i += 1

df = pd.DataFrame({'Image': image_list, 'x': x_list, 'y': y_list, 'width': width_list, 'height': height_list})
df.to_csv(text_file, index=False)

/content/drive/MyDrive/Image Project/1/text/text0.jpg
/content/drive/MyDrive/Image Project/1/text/text1.jpg
/content/drive/MyDrive/Image Project/1/text/text2.jpg
/content/drive/MyDrive/Image Project/1/text/text3.jpg
/content/drive/MyDrive/Image Project/1/text/text4.jpg
